In [1]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer("king")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

{'input_ids': [3364], 'attention_mask': [1]}

In [2]:
tokenizer("queen and the king")

{'input_ids': [4188, 268, 290, 262, 5822], 'attention_mask': [1, 1, 1, 1, 1]}

In [4]:
from transformers import GPT2LMHeadModel

# Generating un-tuned text with our model
#model = GPT2LMHeadModel.from_pretrained("gpt2")

#test_generation = model.generate(max_length=120)
#tokenizer.decode(test_generation[0])

In [5]:
input_sequence = "On a bright sunny day in the streets of Stockholm"
input_ids = tokenizer.encode(input_sequence, return_tensors='pt')

test_generation = model.generate(input_ids, max_length=300)
tokenizer.decode(test_generation[0])

NameError: name 'model' is not defined

In [6]:
input_sequence = "Alf and Yngvi, | Eikinskjaldi"
input_ids = tokenizer.encode(input_sequence, return_tensors='pt')

test_generation = model.generate(input_ids, max_length=120)
tokenizer.decode(test_generation[0])

NameError: name 'model' is not defined

# Loading our dataset

In [9]:
import wget
url = 'https://raw.githubusercontent.com/ohshitnotgood/kth-wrkshp/refs/heads/main/data.txt'
wget.download(url)

100% [.................................] 16438 / 16438

'data.txt'

In [10]:
# Open the file
data = open("./data.txt").read()
print(len(data))

16339


In [11]:
# first, we tokenize the entire dataset
tokenized_text = tokenizer.encode(data, return_tensors="pt")

Token indices sequence length is longer than the specified maximum sequence length for this model (4914 > 1024). Running this sequence through the model will result in indexing errors


In [14]:
examples = []
block_size = 128
for i in range(0, tokenized_text.size(1) - block_size + 1, block_size):
  examples.append(tokenized_text[:, i:i + block_size])

examples[0]

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [15]:
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self):
        self.examples = []
        for i in range(0, tokenized_text.size(1) - block_size + 1, block_size):
            self.examples.append(tokenized_text[:, i:i + block_size])

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx].squeeze()


dataset = TextDataset()
dataloader = DataLoader(dataset, shuffle=True)

In [20]:
# define an optimizer
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=3e-5)

ImportError: cannot import name 'AdamW' from 'transformers' (/Users/yanjun/Documents/apps/miniconda3/envs/dd2421/lib/python3.10/site-packages/transformers/__init__.py)

In [17]:
# Fine-tuning loop
epochs = 3
model.train()
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    for step, batch in enumerate(dataloader):
        batch = batch
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.item():.4f}")


NameError: name 'model' is not defined

In [21]:
# Generate our new text
def generate_text(prompt, max_length=500):
    model.eval()
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(inputs["input_ids"], max_length=max_length, temperature=0.7, do_sample=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Generate text with a prompt
prompt = "Alf and Yngvi, | Eikinskjaldi"

print("Generated text:", generate_text(prompt))

NameError: name 'model' is not defined

In [22]:
for each in range(0, 1000, 128):
  print(each)

0
128
256
384
512
640
768
896
